# LaQuacco 🍅🍅🍅

Laboratory Quality Control v2.0 (2024-09-20)

### User Input

In [ ]:
"""
Basic: Adjust these variable values as needed.
"""
images_path =		"./tests"  # absolute or relative path
images_included =	"*.ome.tiff"  # extension must match
images_excluded =	""  # extension must not match

"""
Advanced: Adjust these variables to customize runtime behavior.
"""
channels_limits = {} # define closed channel intervals with endpoints
copy_images =		False  # increases performance for network drives
plot_c_bands  =		True  # uses group statistics to plot channel bands
recursive_search =	False  # search recursively for image files at path

"""
Check:  Running this cell lists all image files found.
"""
import laquacco as laq
files = laq.get_files(path=images_path,
                       pat=images_included,
                       anti=images_excluded,
                       recurse=recursive_search)
files = [file for file in sorted(files)]
print(f"Images: {len(files)}")
print(f"{[file.replace(images_path, '.') for file in files]}")

print("\nCompleted.")

### Dataset Sampling

In [ ]:
import time

imgs_chans_stats = {}  # main data structure

start = time.time()
update_start = ""
update_stop = ""
update_max = 0
for count, file in enumerate(files, start=1):
    update_len = len(update_start) + len(update_stop)
    update_max =  update_len if update_len > update_max else update_max
    update_start  = f"IMAGE: {file.replace(images_path, '.')}"
    print(f"\r{update_start}{update_stop}{' ':<{update_max - update_len}}", end="")

    image = laq.get_img(file)
    imgs_chans_stats[file] = laq.get_img_chans_stats(image, chans_limits=channels_limits)
    image["tiff"].close()

    update_stop = f" (-{laq.get_time_left(start, count, len(files))})"
print(f"\r{' ' * update_max}\r", end="")  # clear line

print("\nCompleted.")

### Dataset Summary

In [ ]:
import numpy as np

# prepare data
chans = sorted(list(
        set(
            chan
            for chans_stats in imgs_chans_stats.values()
            for chan in chans_stats.keys()
        )))

stats = sorted(
    list(set(
            stat
            for chans_stats in imgs_chans_stats.values()
            for stat in chans_stats.values()
            for stat in stat.keys()
        )))

chans_stats = {
    chan: {
        stat: np.array([imgs_chans_stats[img][chan][stat] for img in imgs_chans_stats])
        for stat in imgs_chans_stats[next(iter(imgs_chans_stats))][chan]
    }
    for chan in imgs_chans_stats[next(iter(imgs_chans_stats))]
}

# print data
for chan in chans:
    print(f"{chan}")
    for stat in stats:
        print(
            f"\t{stat}\t{np.nanmean(chans_stats[chan][stat]):7.1f} (mean)\t{np.nanstd(chans_stats[chan][stat]):7.1f} (std)"
        )
print("\nCompleted.")

### Dataset Distribution - Violin Chart

In [ ]:
import matplotlib as mpl
%matplotlib widget

# prepare figure dimensions
dpi = mpl.pyplot.rcParams["figure.dpi"]
min_pixw, min_pixh = 1600, 900
min_width, min_height = min_pixw / dpi, min_pixh / dpi
mpl.pyplot.rcParams["figure.figsize"] = [min_width, min_height]

# prepare variables for plotting
xy_vals = list(zip(*[(chan, stats["mean"]) for chan, stats in chans_stats.items()]))

# create violin plot
fig, ax = mpl.pyplot.subplots()
ax.set_xticks(range(1, len(xy_vals[0]) + 1), labels=chans, rotation=90)
vp = ax.violinplot(xy_vals[1], showmeans=True, showextrema=False)

# adjust colors
for p in vp["bodies"]:
    p.set_facecolor("black")
    p.set_edgecolor("black")
for m in ["cmeans"]:
    vp[m].set_edgecolor("dimgray")

# add legend
legend = mpl.pyplot.legend(
    [vp["bodies"][0]],
    ["mean"],
)

# adjust axes
mpl.pyplot.minorticks_on()
mpl.pyplot.ylim(bottom=0.0)

# get axes limits
y_min, y_max = mpl.pyplot.gca().get_ylim()

# add Y-axis label
mpl.pyplot.ylabel('Raw values [a.u.]')

# add grid
mpl.pyplot.grid(color='gray', linewidth=0.25)

# show plot
mpl.pyplot.tight_layout()
mpl.pyplot.show()

print("Completed.")

### Dataset Stability - Levey Jennings Charts

In [ ]:
for c, chan in enumerate(chans):
    print(f"\n{chan}:")

    # prepare variables for plotting
    chan_xy_vals = range(len(chans_stats[chan]["mean"])), chans_stats[chan]["mean"]
    chan_mean = np.nanmean(chans_stats[chan]["mean"])
    chan_std = np.nanstd(chans_stats[chan]["mean"])

    # create new plot
    mpl.pyplot.figure()

    # add channel mean
    mpl.pyplot.plot([chan_xy_vals[0][0], chan_xy_vals[0][-1]], [chan_mean, chan_mean], 'r--', label='_', linewidth=1)
    mpl.pyplot.annotate("mean", xy=(-0.1, chan_mean), ha='center', va='center', color='red')
    
    # plot error bars
    mpl.pyplot.errorbar(
        chan_xy_vals[0],
        chan_xy_vals[1],
        fmt="o-",
        linewidth=1,
        markersize=3,
        color="black",
        label=chan,
    )

    # add channel limits
    chan_stds = {}
    chan_stds_keys = [("+2 std", "-2 std"), ("+1 std", "-1 std")]
    chan_stds[chan] = {
        chan_stds_keys[0][0]: chan_mean + 2.0 * chan_std,
        chan_stds_keys[1][0]: chan_mean + 1.0 * chan_std,
        chan_stds_keys[1][1]: chan_mean - 1.0 * chan_std,
        chan_stds_keys[0][1]: chan_mean - 2.0 * chan_std,
    }
    for upper, lower in chan_stds_keys:
        mpl.pyplot.fill_between(
            chan_xy_vals[0],
            chan_stds[chan][upper],
            chan_stds[chan][lower],
            color="black",
            alpha=0.1,
        )
        mpl.pyplot.annotate(upper, xy=(-0.1, chan_stds[chan][upper]), ha='center', va='center', color='darkgray')
        mpl.pyplot.annotate(lower, xy=(-0.1, chan_stds[chan][lower]), ha='center', va='center', color='darkgray')

    # add legend
    mpl.pyplot.legend()
    
    # adjust axes
    mpl.pyplot.minorticks_on()
    
    # add Y-axis label
    mpl.pyplot.ylabel('Raw values [a.u.]')

    # add grid
    mpl.pyplot.grid(color='gray', linewidth=0.25)
    
    # show plot
    mpl.pyplot.tight_layout()
    mpl.pyplot.show()

print("Completed.")